In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np
from sklearn.tree import export_graphviz
import graphviz
import matplotlib.pyplot as plt
import joblib

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/mergeVincesnoAtipicos.csv"
df = pd.read_csv(path, sep=';', low_memory=False)

Mounted at /content/drive


In [3]:
df

,Index,_1,Identificador,year,rc_clacul,ct_prepa_suelo,ct_k510ha,ct_k511ha,ct_afecta_prod,ct_riego,...,pq_herb,ct_color_her_pq,pq_ins,ct_color_ins_pq,pq_fun,ct_color_fun_pq,su_plaguicidas,ct_prod,Ventas,Unnamed: 28
0,168,12678,12085002062560003,2018,507,1,0.0706,0.0706,2,1,...,0.5,4,0.5,1,0.5,4,0.0706,0.272727,0.181818,NaN
1,172,12683,12085002062560004,2018,507,1,0.5000,0.5000,2,1,...,1.0,3,1.0,2,0.0,0,0.5000,4.363636,1.818182,NaN
2,171,12682,12085002062560004,2018,507,1,0.5000,0.5000,2,1,...,1.0,3,1.0,2,0.0,0,0.5000,4.363636,1.818182,NaN
3,170,12681,12085002062560004,2018,507,1,0.5000,0.5000,2,2,...,1.0,3,1.0,2,0.0,0,0.5000,4.363636,1.818182,NaN
4,169,12680,12085002062560004,2018,507,1,0.5000,0.5000,3,2,...,1.0,3,1.0,2,0.0,0,0.5000,4.363636,1.818182,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,163,16815,12085151000993398,2017,507,2,4.9392,4.9392,7,2,...,7.0,3,2.0,4,0.0,0,4.9392,16.838182,16.838182,NaN
475,162,16813,12085151000993398,2017,507,2,9.8784,9.8784,7,2,...,14.0,3,4.0,3,0.0,0,9.8784,33.676364,33.676364,NaN
476,167,16821,12085152000990314,2017,507,1,5.0000,5.0000,7,1,...,0.0,0,2.0,1,9.0,1,5.0000,11.697955,11.697955,NaN
477,166,16820,12085152000990314,2017,507,1,5.0000,5.0000,7,1,...,0.0,0,2.0,1,9.0,1,5.0000,11.697955,11.697955,NaN


PRE-PROCESAMIENTO

In [4]:
df['ct_prod_label'] = pd.cut(df['ct_prod'], bins=[0, 2, 5, 10, np.inf], labels=['malo', 'bueno', 'muy bueno', 'excelente'])

In [5]:
columnas = ['ct_prepa_suelo', 'ct_k510ha', 'ct_k511ha', 'ct_afecta_prod',
                       'ct_riego', 'su_fertilizada', 'ct_fqui', 'ct_fqui_npk',
                       'ct_pqui', 'su_plaguicidas','Ventas'] #Ventas / ct_prod en base a lo que se necesite

In [6]:
X = df[columnas]
y = df['ct_prod_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
df

,Index,_1,Identificador,year,rc_clacul,ct_prepa_suelo,ct_k510ha,ct_k511ha,ct_afecta_prod,ct_riego,...,ct_color_her_pq,pq_ins,ct_color_ins_pq,pq_fun,ct_color_fun_pq,su_plaguicidas,ct_prod,Ventas,Unnamed: 28,ct_prod_label
0,168,12678,12085002062560003,2018,507,1,0.0706,0.0706,2,1,...,4,0.5,1,0.5,4,0.0706,0.272727,0.181818,NaN,malo
1,172,12683,12085002062560004,2018,507,1,0.5000,0.5000,2,1,...,3,1.0,2,0.0,0,0.5000,4.363636,1.818182,NaN,bueno
2,171,12682,12085002062560004,2018,507,1,0.5000,0.5000,2,1,...,3,1.0,2,0.0,0,0.5000,4.363636,1.818182,NaN,bueno
3,170,12681,12085002062560004,2018,507,1,0.5000,0.5000,2,2,...,3,1.0,2,0.0,0,0.5000,4.363636,1.818182,NaN,bueno
4,169,12680,12085002062560004,2018,507,1,0.5000,0.5000,3,2,...,3,1.0,2,0.0,0,0.5000,4.363636,1.818182,NaN,bueno
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,163,16815,12085151000993398,2017,507,2,4.9392,4.9392,7,2,...,3,2.0,4,0.0,0,4.9392,16.838182,16.838182,NaN,excelente
475,162,16813,12085151000993398,2017,507,2,9.8784,9.8784,7,2,...,3,4.0,3,0.0,0,9.8784,33.676364,33.676364,NaN,excelente
476,167,16821,12085152000990314,2017,507,1,5.0000,5.0000,7,1,...,0,2.0,1,9.0,1,5.0000,11.697955,11.697955,NaN,excelente
477,166,16820,12085152000990314,2017,507,1,5.0000,5.0000,7,1,...,0,2.0,1,9.0,1,5.0000,11.697955,11.697955,NaN,excelente


ENTRENAMIENTO DEL MODELO

In [8]:
# Definir los hiperparámetros a ajustar con GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

In [9]:
# modelo RFClassifier
rf = RandomForestClassifier()

In [10]:
# Búsqueda de hiperparámetros
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, error_score='raise')
grid_search.fit(X_train, y_train)

# Modelo con los mejores hiperparámetros encontrados
best_rf = grid_search.best_estimator_

In [11]:
# Realizar predicciones con el conjunto de testing
y_pred = best_rf.predict(X_test)

In [12]:
best_score = grid_search.best_score_
print(best_score)

0.8380041011619959


In [13]:
#valores reales vs predicciones
df2=pd.DataFrame({'Actual':y_test,'Predicted':y_pred})
df2

,Actual,Predicted
93,excelente,excelente
320,malo,malo
55,muy bueno,muy bueno
30,excelente,muy bueno
449,bueno,bueno
...,...,...
249,bueno,bueno
393,bueno,bueno
425,malo,malo
42,excelente,excelente


In [14]:
# Obtener el informe de clasificación
from sklearn.metrics import classification_report
classification_report = classification_report(y_test, y_pred)
print("Informe de clasificación:")
print(classification_report)

Informe de clasificación:
              precision    recall  f1-score   support

       bueno       0.81      0.91      0.85        32
   excelente       0.81      0.81      0.81        16
        malo       0.93      0.89      0.91        28
   muy bueno       0.76      0.65      0.70        20

    accuracy                           0.83        96
   macro avg       0.83      0.82      0.82        96
weighted avg       0.83      0.83      0.83        96



ALMACENAMIENTO DEL MODELO

In [15]:
# Almacendamiento del modelo
hdf5_file = "random_forest_classifier_model.h5"
joblib.dump(best_rf, hdf5_file)
print("El modelo se ha guardado correctamente")

El modelo se ha guardado correctamente


In [16]:
# Cargar el modelo para ser utlizado
hdf5_file = "/content/random_forest_classifier_model.h5"
rf_model = joblib.load(hdf5_file)

MEJOR ARBOL

In [17]:
# Obténcion de las caracteristicas del modelo
feature_importances = rf_model.feature_importances_

# Obtencion del indice con respecto a la caracteristica mas importante
best_feature_index = np.argmax(feature_importances)

# Mejor árbol relacionado a la característica más importante
tree = rf_model.estimators_[best_feature_index]

In [18]:
!pip install dtreeviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 5.3 MB/s eta 0:00:00


In [21]:
tree = best_rf.estimators_[0]
feature_names = X_train.columns.tolist()
# Exporta el árbol en formato DOT
dot_data = export_graphviz(tree, out_file=None,feature_names= feature_names,
                           filled=True, rounded=True)

# Crea un objeto de gráfico a partir de los datos DOT
graph = graphviz.Source(dot_data)

# Muestra el gráfico del árbol
graph.view()


'Source.gv.pdf'